In [22]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from requests import *
import json

In [3]:
client = dataiku.api_client()

In [29]:
dss_version = dataiku.get_custom_variables()['dss_version']

params = dict(
    meta = ['label', 'description', 'author', 'url', 'tags', 'supportLevel'],
    default = ['storeVersion','downloadURL', 'size', 'content']
)

In [23]:
url = "https://update.dataiku.com/dss/{}/plugins/list.json".format(dss_version)

In [31]:
page_request = json.loads(request('GET', url).content)['items']

In [32]:
def get_plugin_meta(item, params):

    plugin_meta = {}

    for k,v in params.items():

        if k=='default':
            for e in v:
                if e in item.keys():
                    plugin_meta[e] = item[e]
                else:
                    plugin_meta[e] = np.nan
        else:
            for e in v:
                if k in item.keys() and e in item[k].keys():
                    plugin_meta[e] = item[k][e]
                else:
                    plugin_meta[e] = np.nan
    return plugin_meta

items = {}

for i, item in enumerate(page_request):

    plugin_meta = get_plugin_meta(item, params)
    plugin_meta['release_time']= max([item['revisions'][i]['releaseTime'] for  i in range(len(item['revisions']))])

    items[item['id']] = plugin_meta

df_plugins = pd.DataFrame.from_dict(items,orient='index').reset_index().rename(columns={'index':'pluginId'})

In [33]:
df_plugins

pluginId                                        label                                        description                                             author                                                url                                               tags       supportLevel storeVersion                                        downloadURL       size                                            content   release_time
0                       ab-test-calculator                           AB test calculator           Design and analyse an AB test experiment                 Dataiku (Marine SOBAS and Du PHAN)  https://www.dataiku.com/dss/plugins/info/ab-te...                                        [Marketing]      NOT_SUPPORTED        1.0.1  https://update.dataiku.com/dss/11/plugins/ab-t...      51589  {'customParameterSets': [], 'customRNotebookTe...  1652708756969
1                                 airtable                                     Airtable  Fetch data from Airtable, a tool that provides...                             Dataiku (Jérémy Grèze)  https://www.dataiku.com/dss/plugins/info/airta...                                        [Connector]                NaN        0.1.4  https://update.dataiku.com/dss/11/plugins/airt...       4393  {'customParameterSets': [], 'customRNotebookTe...  1652708750638
2                             aks-clusters                                 AKS clusters  Interact with or create Microsoft Azure Kubern...                                            Dataiku  https://github.com/dataiku/dss-plugin-aks-clus...                           [Cloud, Microsoft Azure]      TIER2_SUPPORT        2.1.0  https://update.dataiku.com/dss/11/plugins/aks-...      48737  {'customParameterSets': [{'meta': {'icon': 'ic...  1668081441000
3                            algoliasearch                     Algolia Search Connector  A read-and-write connector for the Algolia sea...                           Dataiku (Clement Stenac)  https://www.dataiku.com/dss/plugins/info/algol...                                        [Connector]      NOT_SUPPORTED        0.0.9  https://update.dataiku.com/dss/11/plugins/algo...       3622  {'customParameterSets': [], 'customRNotebookTe...  1652708756659
4            amazon-comprehend-nlp-medical                    Amazon Comprehend Medical  Use Amazon Comprehend Medical APIs to extract ...                           Dataiku (Alex COMBESSIE)  https://www.dataiku.com/product/plugins/amazon...                                  [AWS, Cloud, NLP]      TIER2_SUPPORT        1.0.2  https://update.dataiku.com/dss/11/plugins/amaz...      27794  {'customParameterSets': [{'meta': {'icon': 'ic...  1652708756867
5                    amazon-comprehend-nlp                        Amazon Comprehend NLP  Use Amazon Comprehend APIs to perform Language...          Dataiku (Alex COMBESSIE, Joachim ZENTICI)  https://www.dataiku.com/product/plugins/amazon...                                  [AWS, Cloud, NLP]      TIER2_SUPPORT        1.0.1  https://update.dataiku.com/dss/11/plugins/amaz...      34660  {'customParameterSets': [{'meta': {'icon': 'ic...  1652708756864
6                       amazon-rekognition                           Amazon Rekognition  Use Amazon Rekognition APIs to perform Object ...          Dataiku (Alex COMBESSIE, Joachim ZENTICI)  https://www.dataiku.com/product/plugins/amazon...               [AWS, Cloud, Image, Computer Vision]      TIER2_SUPPORT        1.0.4  https://update.dataiku.com/dss/11/plugins/amaz...     156171  {'customParameterSets': [{'meta': {'icon': 'ic...  1652708756911
7                        amazon-transcribe                            Amazon Transcribe  Amazon Transcribe plugin. The plugin takes aud...                         Dataiku (Jeremy PLASSMANN)  https://www.dataiku.com/product/plugins/amazon...                                  [Cloud, AWS, NLP]      NOT_SUPPORTED        1.0.1  https://update.dataiku.com/dss/11/plugins/amaz...      67629  {'customParameterSets': [{'meta

In [0]:
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

plugins_store_df = ... # Compute a Pandas dataframe to write into plugins_store


# Write recipe outputs
plugins_store = dataiku.Dataset("plugins_store")
plugins_store.write_with_schema(plugins_store_df)